# Introduction to the pairinteraction Library

In addition to the [graphical user interface](https://pairinteraction.github.io/images/screen-win64.png), the pairinteraction software includes a library. The library can be used to write your own code and have more fine-grained control over what pairinteraction does, e.g. for searching optimal experimental parameters or calculating effective Hamiltonians and simulating Rydberg experiments.

The library is fully written in C++ to obtain high performance. It provides a Python API generated with [SWIG](http://www.swig.org) so that one can work with all the functionality of the library in Python. The Python functions mirror the eponymous wrapped C++ functions. Thus, it is straight forward to transfer code between the two programming languages. The following introduction, which can be [downloaded](https://github.com/pairinteraction/pairinteraction/blob/master/doc/sphinx/examples_python/introduction.ipynb) as a Jupyter notebook, shows the basic usage of the pairinteraction library in Python 3. The physics behind the presented calculations is reviewed in the pairinteraction paper: S. Weber, C. Tresp, H. Menke, A. Urvoy, O. Firstenberg, H. P. Büchler, S. Hofferberth, [J. Phys. B: At. Mol. Opt. Phys. 50, 133001 (2017)](https://doi.org/10.1088/1361-6455/aa743a) ([arXiv:1612.08053](https://arxiv.org/abs/1612.08053)).

Note that, in the unit system used by the pairinteraction software, energy has the unit $\text{GHz}$, length $\mu\text{m}$, the magnetic field $\text{G}$, and the electric field $\text{V/cm}$.

## Preparations

### Installation
By installing a [binary build of the pairinteraction software](https://github.com/pairinteraction/pairinteraction/releases), the pairinteraction Python library gets installed, too. For GNU/Linux, all dependencies of the pairinteraction library are installed automatically as well, and the library can be used right away. For Windows or OS X, we recommend the installation of the Python 3 distribution [Anaconda](https://www.anaconda.com/distribution/). Then, the dependencies can be installed by executing ``conda install numpy scipy matplotlib pyzmq`` within the Anaconda terminal prompt. Its important that the installed version of the `numpy` library is up-to-date. 

### Importing the Library
Our code starts with loading the required modules for the calculations. We use the module `pireal` of the pairinteraction library as the calculations shown in this introduction require only real numbers (if one considers electric or magnetic fields with a non-zero $y$-value, complex numbers occur due to the definition of the spherical basis and one must use `picomplex`).

In [1]:
# We call an IPython magic function to make the output of plotting commands displayed inline within this notebook.
%matplotlib inline

# Arrays
import numpy as np

# Plotting
import matplotlib.pyplot as plt

# Operating system interfaces
import os, sys

# pairinteraction :-)
# For Windows and OS X, we manually add the path containing the library to the Python package search path.
#if sys.platform == "darwin": sys.path.append("/Applications/pairinteraction.app/Contents/Resources")
#elif sys.platform == "win32": sys.path.append("C:\Program Files\pairinteraction")
sys.path.append("/Users/sebastian/Desktop/03-Pairinteraction/01-repositories/pairinteraction-fork-new2/build")
from libpairinteraction import pireal as pi

### Creating a Cache for Matrix Elements

The `MatrixElementCache` class provides methods for evaluating matrix elements. Each Python script using the pairinteraction library typically requires one instance of this class. The instance is then passed to every object of the pairinteraction library which needs to evaluates matrix elements. To speed up calculations, the intermediate results of the calculation of matrix elements are cached into memory. If a directory name is passed to the constructor of the class, the specified directory is used to store a SQLite database which holds the intermediate results, making them available to future program runs.

In [2]:
if not os.path.exists("./cache"):
    os.makedirs("./cache")
cache = pi.MatrixElementCache("./cache")

## Defining States
Rydberg states are defined in the fine structure basis by specifying the *species* and the quantum numbers $n$, $l$, $j$, $m_j$. The pairinteraction software natively supports as species the  **alkali metals** lithium (`"Li"`), sodium (`"Na"`), potassium (`"K"`), rubidium (`"Rb"`), caesium (`"Cs"`). In addition, the **alkaline earth metal** strontium is supported in its singlet (`"Sr1"`) and triplet state (`"Sr3"`). The species-specific quantum defects and model potential parameters are stored in a database, created from a [SQL file](https://github.com/pairinteraction/pairinteraction/blob/master/libpairinteraction/databases/quantum_defects.sql). Note that we do not differentiate between isotopes as they possess nearly identical quantum defects.

The user can add further species to pairinteraction by inserting their quantum defects and model potential parameters into the SQL file and creating a new database. In order to make pairinteraction use the new database, its path must be passed to the `MatrixElementCache` object by executing `cache.setDefectDB("path/to/database.db")`.

[//]: # (TODO: implement setDefectDB, calling this method should disable caching to the hard drive)

### Single Atom States
The `StateOne` class allows for defining states of single Rydberg atoms. For example, the state with the quantum numbers $n=61$, $l=0$, $j=1/2$, $m_j=-1/2$ of a rubidium atom is written as:

In [3]:
state = pi.StateOne("Rb", 61, 0, 0.5, -0.5)

### Pair States
The `StateTwo` class allows for defining states of two Rydberg atoms. Such a pair state can be defined as a combination of two single atom states.

In [4]:
state1 = pi.StateOne("Rb", 61, 0, 0.5, -0.5)
state2 = pi.StateOne("Cs", 60, 1, 1.5, 1.5)
state = pi.StateTwo(state1, state2)

Alternatively, a pair state can be initialized by specifying all parameters in pairs.

In [5]:
state = pi.StateTwo(["Rb", "Cs"], [61, 60], [0, 1], [0.5, 1.5], [-0.5, 1.5])

Afte the initialization of a pair state, the state of the first atom can be obtained by `StateTwo.getFirstState()` and the state of the second atom by `StateTwo.getSecondState()`.

The classes `StateOne` and `StateTwo` own methods for receiving the properties of the states. Species are obtained by `State[...].getSpecies()` and quantum numbers by `State[...].getN()`, `State[...].getL()`, `State[...].getJ()`, `State[...].getM()`.

[//]: # (TODO: implement missing functionality)

## Application 1: Energy Levels and Effective Principal Quantum Numbers

The energy of a single atom state or the total energy of a pair state is received by calling the method `State[...].getEnergy()`. Similarly, effective principal quantum numbers $n^*$ are obtained by `State[...].getNStar()`.

[//]: # (TODO: pretify output of print command, it should be |Rb, 61 S_1/2, mj=1/2>)

In [6]:
# Define Rydberg state
state = pi.StateOne("Rb", 61, 0, 0.5, 0.5)

# Get the energy of the state
print("The energy of {} is {} GHz.".format(state, state.getEnergy()))

# Get the effective principal quantum number of the state
print("The effective principal quantum number of {} is {}.".format(state, state.getNStar()))

The energy of n  = 61,   l  =  0,   j  =  0.5,   m  =  0.5 is -982.3898169877917 GHz.
The effective principal quantum number of n  = 61,   l  =  0,   j  =  0.5,   m  =  0.5 is 57.86876593760547.


## Application 2: Matrix Elements

[//]: # (TODO: discuss the units of the matrix elements, calculate LeRoy radius as an example)

The previously created instance of the `MatrixElementCache` class can be applied directly to calculate matrix elements $\langle f \rvert A_q \lvert i \rangle$, where $A_q$ is an operator in spherical coordinates, $\lvert i \rangle=\lvert n',l',j',m_j'\rangle$ is the initial Rydberg state, and $\langle f\rvert =\langle n, l, j, m \rvert$ the final Rydberg state. The order $q$ of the operator $A_q$ is assumed to equal $m_j-m_j'$. The following matrix elements are supported:

- `MatrixElementCache.getElectricDipole(state_f, state_i)` returns the matrix element of the **electric dipole operator** 
$d_q = \sqrt{\frac{4\pi}{3}} e r Y_{1q}$ in units of $\text{GHz}^1(\text{V}/\text{cm})^{-1}$, so that $\langle f \rvert d~E \lvert i \rangle$ and $\langle f \rvert \frac{d~d}{4\pi\epsilon_0 R^3} \lvert i \rangle$ have the unit of an energy in the unit system used by the pairinteraction software.
- `MatrixElementCache.getElectricMultipole(state_f, state_i, kappa_radial, kappa_angular)` returns the matrix element of a generalized form of the **electric multipole operator** $p_{\kappa_\text{radial}, \kappa_\text{angular},q} = \sqrt{\frac{4\pi}{2\kappa_\text{angular}+1}} e r^{\kappa_\text{radial}} Y_{\kappa_\text{angular} q}$
 in units of $\text{GHz}^1(\text{V}/\text{cm})^{-1}\mu\text{m}^{\kappa_\text{radial}-1}$.
- `MatrixElementCache.getMagneticDipole(state_f, state_i)` returns the matrix element of the **magnetic dipole operator** $\mu_q = - \frac{\mu_B}{\hbar} (g_l l_q + g_s s_q)$ in units of $GHz^1 G^{-1}$, so that $\langle f \rvert \mu ~B \lvert i \rangle$ has the unit of an energy in the used unit system.
- `MatrixElementCache.getRadial(state_f, state_i, kappa)` returns the matrix element of the **radial operator** $r^\kappa$ in units of $\mu\text{m}^\kappa$.

The calculation of matrix elements of more complex operators can often be reduced to the calculation of the matrix elements stated above. Sometimes, additional constants occure in the expressions of the operators. The pairinteraction library provides the following constants:

- `coulombs_constant` with value $1/(4\pi\epsilon_0) = 0.5955214763029308~\text{GHz}^{-1}(\text{V}/\text{cm})^2 \mu\text{m}^3$
- `electron_rest_mass` with value $m_e = 1374779.2437085041~\text{GHz}^1(\text{V}/\text{cm})^{-2}\text{G}^2$
- `elementary_charge` with value $e = 24.17989262349962~\text{GHz}^1(\text{V}/\text{cm})^{-1}\mu \text{m}^{-1}$

As an example, we show how to calculate a matrix element of the dipole-dipole interaction operator $V_{dd} = \frac{-2d_0d_0-d_+d_- - d_-d_+}{4 \pi \epsilon_0 R^3}$.

[//]: # (TODO: link applied unit system - make an extra section, functions for returning Coulomb's constant etc, adapt methods to fit the description, give link to further example, explain getFirstState and getSecondState in the pair state section, format list differently so that it is shown correctly in the documentation, pi.PerturbativeDipolarInteraction # TODO check whether theta == 0)

In [47]:
# Function for calculating a matrix element of the dipole-dipole interaction operator
def getDipoleDipole(state_f, state_i, distance):
    q = state_f.getM()-state_i.getM()
    
    if q[0] == 0 and q[1] == 0:
        prefactor = -2
    elif q[0] == 1 and q[1] == -1:
        prefactor = -1
    elif q[0] == -1 and q[1] == 1:
        prefactor = -1
    else:
        return 0
    
    return prefactor * pi.coulombs_constant / distance**3 * \
        cache.getDipole(state_f.getFirstState(), state_i.getFirstState()) * \
        cache.getDipole(state_f.getSecondState(), state_i.getSecondState()) 

# Define Rydberg states
state_i = pi.StateTwo(["Rb", "Rb"], [61, 61], [0, 1], [0.5, 0.5], [0.5, 0.5])
state_f = pi.StateTwo(["Rb", "Rb"], [61, 61], [1, 0], [0.5, 0.5], [0.5, 0.5])

# Get the matrix element at an interatomic distance of 10 um
distance = 10 # um
print("The matrix element has the value {} GHz.".format(getDipoleDipole(state_f, state_i, distance)))

The matrix element has the value -0.003252636908333978 GHz.


## Application 3: Dispersion Coefficients

We consider two interacting Rydberg atoms. We call the interatomic distance $R$ and the angle between the interatomic axis and the quantization axis the interaction angle $\theta$. At large interatomic distances, the energy shifts of  Rydberg pair states due to the interaction can be estimated perturbatively.

### Non-degenerate States

For the beginning, we assume that the Rydberg pair state $\lvert rr \rangle$, for which we calculate the energy shift, has no degenerate states it can couple to. Thus *second order non-degenerate perturbation theory* is applicable and the energy shift is $\Delta E = C_6 / R^6$, where $C_6$ is the dispersion coefficient of the van der Waals interaction. For the calculation of the $C_6$ coefficient, we only consider states that couple significantly to $\lvert rr \rangle$. We ensure this by requiring that the difference between the principal quantum numbers of $\lvert rr \rangle$ and of the considered states is less than or equal to a constant $\Delta N$, which can be set by the user to achieve convergence.

In [58]:
# Define Rydberg state for which the C6 coefficient should be calculated
state = pi.StateTwo(["Cs", "Cs"], [42, 42], [0, 0], [0.5, 0.5], [0.5, 0.5])

# Angle between the interatomic axis and the quantization axis
theta = 0 # rad

# Use only states with similar principal quantum numbers for the calculation
deltaN = 3

# Get the C6 coefficient
calculator = pi.PerturbativeDipolarInteraction(cache, theta)
print("The C6 coefficient is {} GHz um^6.".format(calculator.getC6(state, deltaN)))

The C6 coefficient is 1.2105362290320056 GHz um^6.


### Degenerate States

In case of degenerate states, we must take into account the full subspace of degenerate states and use degenerate perturbation theory. Instead of a scalar dispersion coefficient, we obtain a matrix of the dimension of the degenerate subspace. If the states within the degenerate subspace couple in second order, we are still in the van der Waals regime and must apply *second order degenerate perturbation theory*, using the method `PerturbativeDipolarInteraction.getC6(degenerate_states, deltaN)` (the returned matrix has the unit $\text{GHz}\,\mu\text{m}^6$). If the states within the degenerate subspace couple directly, we are in the resonant dipole-dipole regime and must apply *first order degenerate perturbation theory*, using `PerturbativeDipolarInteraction.getC3(degenerate_states)` (the returned matrix has the unit $\text{GHz}\,\mu\text{m}^3$).

In [109]:
# Define degenerate subspace of Rydberg states that couple in second order
degenerate_states = [pi.StateTwo(["Cs", "Cs"], [42, 42], [0, 0], [0.5, 0.5], [m1, m2]) \
                     for m1 in [-0.5, 0.5] for m2 in [-0.5, 0.5]]
print("Basis of the degenerate subspace:")
print('\n'.join(str(state) for state in degenerate_states))

# Angle between the interatomic axis and the quantization axis
theta = np.pi/3 # rad

# Use only states with similar principal quantum numbers for the calculation
deltaN = 3

# Get the C6 matrix
calculator = pi.PerturbativeDipolarInteraction(cache, theta)
print("\nC6 matrix in the basis of the degenerate subspace, in GHz um^6:")
print(np.round(calculator.getC6(degenerate_states, deltaN),2))

Basis of the degenerate subspace:
n0 = 42,   l0 =  0,   j0 =  0.5,   m0 = -0.5,   n1 = 42,   l1 =  0,   j1 =  0.5,   m1 = -0.5
n0 = 42,   l0 =  0,   j0 =  0.5,   m0 = -0.5,   n1 = 42,   l1 =  0,   j1 =  0.5,   m1 =  0.5
n0 = 42,   l0 =  0,   j0 =  0.5,   m0 =  0.5,   n1 = 42,   l1 =  0,   j1 =  0.5,   m1 = -0.5
n0 = 42,   l0 =  0,   j0 =  0.5,   m0 =  0.5,   n1 = 42,   l1 =  0,   j1 =  0.5,   m1 =  0.5

C6 matrix in the basis of the degenerate subspace, in GHz um^6:
[[ 1.25  0.02  0.02 -0.04]
 [ 0.02  1.21  0.03 -0.02]
 [ 0.02  0.03  1.21 -0.02]
 [-0.04 -0.02 -0.02  1.25]]


## Application 4: Non-perturbative Calculations

In many recent Rydberg experiments, the measurements are so precise that deviations from the perturbative description are getting significant. In addition, there are Rydberg systems where perturbative calculations are not working at all because splittings between energy levels are smaller than interaction energies.  This especially happens for short interatomic distances or in the presence of electric fields. To study these systems, we must diagonalize their Hamiltonians.

We can apply the pairinteraction library to construct and diagonalize Hamiltonians of Rydberg systems. For systems consisting of one single Rydberg atom, the class `SystemOne` is provided. For two Rydberg atoms, the class `SystemTwo` is given. Both of these classes are used in a similar way:

- First, we tell which Rydberg states should be considered, letting the software create a **list of relevant Rydberg states**. The Rydberg states can be obtained by `System[...].getStates()`.
- Second, we specify whether symmetries of the systems should be taken into account to speed up calculations. The software generates a **list of basis vectors**, where each basis vector is a linear combination of the previously given Rydberg states. If symmetries are specified, they are applied to reduce the size of the basis. The basis vectors can be obtained as the columns of the matrix returned by `System[...].getBasis()`.
- Third, we set up the interactions. The software builds a **matrix representation of the Hamiltonian** in the previously generated basis. The Hamiltonian can be obtained by `System[...].getHamiltonian()`.

After calling the method `System[...].diagonalize()`, the diagonal entries of the Hamiltonian contain the eigenenergies of the system and the list of basis vectors contains the eigenvectors.

[//]: # (TODO: rename the methods specified in SystemBase to meet the nomenclature used in this Jupyter notebook, check whether the columns or rows of System[...].getBasis are the basis vectors)

### Systems Consisting of One Rydberg Atom, Calculating Stark and Zeeman Maps

*[Text will be added soon.]*

### Systems Consisting of Two Rydberg Atoms, Calculating Pair Potentials

*[Text will be added soon.]*